# Lab | Data cleaning and wrangling

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs. 

### Instructions

So far we have worked on `EDA`. This lab will focus on data cleaning and wrangling from everything we noticed before.

1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.
2. Create a copy of the dataframe for the data wrangling.
3. Normalize the continuous variables. You can use any one method you want.
4. Encode the categorical variables
5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

**Hint for Categorical Variables**

- You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

```python
# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
# given that column "coverage" in the dataframe "data" has three categories:
# "basic", "extended", and "premium" and values are to be represented in the same order.
```


### LAB Solution

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
customer_df = pd.read_csv("./files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")
customer_df.columns = [col.lower().replace(' ', '_')  for col in customer_df.columns]

In [15]:
# No NaNs so just remove outliers (normalization method)
def outliers(column, threshold = 3):
    """
    docs
    """

    data = column[abs(column.apply(lambda x: (x - column.mean()) / column.var() ** (1/2))) > threshold]
    return data

In [16]:
CLV_outliers = outliers(customer_df["customer_lifetime_value"])
MPA_outliers = outliers(customer_df["monthly_premium_auto"])

In [20]:
to_drop = CLV_outliers.index | MPA_outliers.index # Union
clean_customer_df = customer_df.drop(to_drop).reset_index(drop = True)
clean_customer_df.head(3)

C:\Users\EB885UN\AppData\Local\Temp\ipykernel_16916\3911555071.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  to_drop = CLV_outliers.index | MPA_outliers.index # Union


,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize


In [18]:
df_copy = clean_customer_df.copy()

In [46]:
# Normalizing continuous variables (but target)

continuous = customer_df.select_dtypes(include=['int64','float']).columns
continuous = continuous.tolist()

continuous.remove("months_since_policy_inception")
continuous.remove("total_claim_amount")

for cont_var in continuous:
    maximum = clean_customer_df[cont_var].max()
    minimum = clean_customer_df[cont_var].min()
    clean_customer_df[cont_var] = clean_customer_df[cont_var].apply(lambda x: (x - minimum) / (maximum - minimum))
clean_customer_df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,0.032522,No,Basic,Bachelor,2/24/11,Employed,F,0.562847,...,0.050505,0.0,0.000,Corporate Auto,Corporate L3,Offer1,Agent,0.165321,Two-Door Car,Medsize
1,QZ44356,Arizona,0.190941,No,Extended,Bachelor,1/31/11,Unemployed,F,0.000000,...,0.424242,0.0,0.875,Personal Auto,Personal L3,Offer3,Agent,0.486177,Four-Door Car,Medsize
2,AI49188,Nevada,0.412934,No,Premium,Bachelor,2/19/11,Employed,F,0.487763,...,0.383838,0.0,0.125,Personal Auto,Personal L3,Offer1,Agent,0.243385,Two-Door Car,Medsize
3,WW63253,California,0.215979,No,Basic,Bachelor,1/20/11,Unemployed,M,0.000000,...,0.656566,0.0,0.750,Corporate Auto,Corporate L2,Offer1,Call Center,0.227661,SUV,Medsize
4,HB64268,Washington,0.034407,No,Basic,Bachelor,2/3/11,Employed,M,0.438443,...,0.444444,0.0,0.000,Personal Auto,Personal L1,Offer1,Agent,0.059316,Four-Door Car,Medsize


**REMEMBER**

- Education, employment status, policy, and vehicle class are somewhat unbalanced.
- For education, we could turn it into a binary variable (`college +-`), but I wouldn't touch it.
- The policy is redundant, maybe we can classify it in `L1`, `L2` and `L3` groups.
- Id concatenates luxury SUV, sports car and luxury car into luxury or among the other classes.
- For employment, we could divide them among employed, unemployed and inactive.
- We can see that having open complaints isn't that common, so we can turn it into a binary variable, open - not open.
- For the number of policies, we could join use 1, 2, 3, 4+.

In [47]:

# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

customer_df.isna().sum()/len(customer_df)
clean_customer_df["education"] = clean_customer_df["education"].apply(lambda x: "Graduate" if x in ["Master", "Doctor"] else x)
inactive = ["Medical Leave", "Disabled", "Retired"]

In [48]:
clean_customer_df["employmentstatus"] = clean_customer_df["employmentstatus"].apply(lambda x: "Inactive" if x in inactive else x)
clean_customer_df["gender"] = clean_customer_df["gender"].apply(lambda x: 1 if x == "F" else 0)
clean_customer_df["policy"] = clean_customer_df["policy"].apply(lambda x: x[-2:])
luxury = ["Sports Car", "Luxury SUV", "Luxury Car"]
clean_customer_df["vehicle_class"] = clean_customer_df["vehicle_class"].apply(lambda x: "Luxury" if x in luxury else x)

In [49]:
# copy
final_df = clean_customer_df.copy()

In [53]:
# drop customer (id)
ordinal = clean_customer_df.drop(columns = "customer")

In [54]:
# Ordinal encoders
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# Ordinal vehicle size

ordinal["coverage"] = ordinal["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
ordinal["employmentstatus"] = ordinal["employmentstatus"].map({"Unemployed" : 0, "Inactive" : 1, "Employed" : 2})
ordinal["location_code"] = ordinal["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
ordinal["vehicle_size"] = ordinal["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [59]:
one_hot = ordinal.copy()
one_hot_colums = final_df.select_dtypes(include = object).columns[1:]
one_hot_colums

# One hot encoders

# One hot to state
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class

Index(['state', 'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'location_code', 'marital_status', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'vehicle_class',
       'vehicle_size'],
      dtype='object')

In [60]:
one_hot = pd.get_dummies(one_hot, columns = one_hot_colums)

In [62]:
final_df = one_hot.copy()
#final_df["day"] = time_df["day"]
#final_df["week"] = time_df["week"]
#final_df["month"] = time_df["month"]
#final_df = final_df.drop(columns = "effective_to_date")
final_df.apply(pd.to_numeric)

,customer_lifetime_value,gender,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,state_Arizona,...,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Four-Door Car,vehicle_class_Luxury,vehicle_class_SUV,vehicle_class_Two-Door Car,vehicle_size_0,vehicle_size_1,vehicle_size_2
0,0.032522,1,0.562847,0.059259,0.914286,0.050505,0.0,0.000,0.165321,0,...,0,0,0,0,0,0,1,0,1,0
1,0.190941,1,0.000000,0.244444,0.371429,0.424242,0.0,0.875,0.486177,1,...,0,0,0,1,0,0,0,0,1,0
2,0.412934,1,0.487763,0.348148,0.514286,0.383838,0.0,0.125,0.243385,0,...,0,0,0,0,0,0,1,0,1,0
3,0.215979,0,0.000000,0.333333,0.514286,0.656566,0.0,0.750,0.227661,0,...,0,1,0,0,0,1,0,0,1,0
4,0.034407,0,0.438443,0.088889,0.342857,0.444444,0.0,0.000,0.059316,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,0.808175,0,0.719547,0.088889,0.514286,0.898990,0.0,0.125,0.085144,0,...,0,0,1,1,0,0,0,0,1,0
8749,0.045034,1,0.216081,0.133333,0.400000,0.282828,0.0,0.000,0.162909,0,...,1,0,0,1,0,0,0,0,1,0
8750,0.235444,0,0.000000,0.177778,0.257143,0.373737,0.6,0.125,0.339778,0,...,1,0,0,1,0,0,0,0,1,0
8751,0.211417,0,0.219452,0.259259,0.971429,0.030303,0.0,0.250,0.296984,0,...,1,0,0,1,0,0,0,0,0,1


### Additional Resources

- [Difference between `standardscaler` and Normalizer in `sklearn.preprocessing`](https://stackoverflow.com/questions/39120942/difference-between-standardscaler-and-normalizer-in-sklearn-preprocessing)